In [10]:
import numpy as np
from PIL import Image
import torch
from torch.nn import Unfold # this is PyTorch's Im2Col function


def w_idx2a_idx(Ashape, tshape, i):
    n,m = Ashape
    h,l = tshape
    
    n_rows = m - l + 1
    return np.array([i // n_rows, i % n_rows])

# Hiding a photo within another

This code takes a target image and finds the optimal hiding spot for it within another photo. This project was inspired by the [Nature Briefing](https://www.nature.com/nature/articles?type=nature-briefing). 

For more details, see the related blog post on my [website](https://nmarzz.github.io/)

In [80]:
###### Import and adjust our target image (The image to hide)

## We'll scale down our base and target image for efficiency
base_scale = 10
target_scale = base_scale

# We can slightly resize our penguin image to hide it better
penguin_scale = 1
penguin_size = np.round(penguin_scale * np.array([40, 80])).astype(int)

t_im = Image.open('rockhopper.png')
# t_im = t_im.transpose(Image.FLIP_LEFT_RIGHT)
t_im = t_im.convert('RGBA')
t_im = t_im.resize(list(penguin_size)) # Scale the penguin down to hidding size

# We'll also try hiding the flipped image
t_im_flip = t_im.transpose(Image.FLIP_LEFT_RIGHT)


t_im_resized = t_im.resize(np.array(t_im.size) // (target_scale)) # scale down for efficiency
t_im_flip_resized = t_im_flip.resize(np.array(t_im.size) // (target_scale))


# Form our target matrix for original t
t = np.array(t_im_resized).transpose()
mask = (np.concatenate([t[-1,:,:].flatten(),t[-1,:,:].flatten(),t[-1,:,:].flatten()]) == 0) # Accounting for Leif's non-rectangularity

t = torch.tensor(t[0:3,:,:], dtype = torch.float32)
flat_t = t.flatten()
t_size = (t.shape[1], t.shape[2])


# Form our target matrix for flipped t
t_flip = np.array(t_im_flip_resized).transpose()
mask_flip = (np.concatenate([t_flip[-1,:,:].flatten(),t_flip[-1,:,:].flatten(),t_flip[-1,:,:].flatten()]) == 0) # Accounting for Leif's non-rectangularity

t_flip = torch.tensor(t_flip[0:3,:,:], dtype = torch.float32)
flat_t_flip = t_flip.flatten()


###### Import and adjust our base Image 
base = Image.open('sunset.jpeg')
base = base.convert('RGBA')
base_resized = base.resize(np.array(base.size) // base_scale)

# Form our base matrix
A = np.array(base_resized).transpose()
A = torch.tensor(A, dtype = torch.float32)
A = torch.unsqueeze(A,0)
A = A[:,0:3,:,:]

In [81]:
# Calculate W with im2col
unfold = Unfold(kernel_size= t_size)
W = unfold(A).squeeze(0).transpose(0,1)

In [82]:
# Calculate the distance between each patch and our original target 
difference = W - t.flatten()
difference[:,mask] = 0 # Ignore the masked out entries
norms = np.linalg.norm(difference, axis = 1)

min_norm = np.min(norms)
optimal_window_idx = np.argmin(norms)

In [83]:
# Calculate the distance between each patch and our original target 
difference = W - t_flip.flatten()
difference[:,mask_flip] = 0 # Ignore the masked out entries
norms = np.linalg.norm(difference, axis = 1)

min_norm_flip = np.min(norms)
optimal_window_idx_flip = np.argmin(norms)

In [84]:
# Find the location of the best patch 
if min_norm <= min_norm_flip:
    optimal_index = w_idx2a_idx((A.shape[2],A.shape[3]), (t.shape[1], t.shape[2]) ,optimal_window_idx)
    paste_im = t_im
    print('Better orientation: original')
else:
    optimal_index = w_idx2a_idx((A.shape[2],A.shape[3]), (t.shape[1], t.shape[2]) ,optimal_window_idx_flip)
    paste_im = t_im_flip    
    print('Better orientation: flipped')    
print(f'Optimal index is {optimal_index * base_scale}')

Better orientation: flipped
Optimal index is [220  20]


In [85]:
# Hide our target and display!
base.paste(paste_im, tuple(optimal_index * base_scale) ,  mask=paste_im) 
base.show()